In [1]:
# import required modules
import pandas as pd
import os

# get renumbering key
dfRenumber = pd.read_csv(r"_node_key_v832_to_v9_bh.csv")
dfRenumber

,N,N_V9,N_V832_error,N_v832_correct,description
0,8,1,8,8,GEOGKEY matches
1,9,2,9,9,GEOGKEY matches
2,2,3,2,2,GEOGKEY matches
3,1,4,1,1,GEOGKEY matches
4,10,5,10,10,GEOGKEY matches
...,...,...,...,...,...
22080,2877,102877,2877,2877,Moved Centroid Node
22081,2878,102878,2878,2878,Moved Centroid Node
22082,2879,102879,2879,2879,Moved Centroid Node
22083,2880,102880,2880,2880,Moved Centroid Node


In [2]:
# select specific folder year
#linYear = "Lin_2040_Needs"

# assign directories
directory = os.getcwd()
#NewNumDirectory = os.path.normpath(os.getcwd() + os.sep + os.pardir + "\Transit-v9\\" + linYear)
#OldNumDirectory = os.path.normpath(os.getcwd() + os.sep + os.pardir + "\Transit-v832\\" + linYear)

OldNumDirectory = "E:\\GitHub\\TDM-Transit-Node-Renumbering-v832-to-v9\\Transit-MAG-2050FC\\Old Numbers"
NewNumDirectory = "E:\\GitHub\\TDM-Transit-Node-Renumbering-v832-to-v9\\Transit-MAG-2050FC\\New Numbers"

In [3]:
# iterate over files in v832 directory of selected year
for filename in os.listdir(OldNumDirectory):
  f = os.path.join(OldNumDirectory, filename)
  # checking if it is a file
  if os.path.isfile(f) & f.endswith('.lin'):
    print("")
    print("FILE: " + f)

    strFilename = os.path.join(NewNumDirectory + '\\' + os.path.basename(f));
    with open(strFilename, 'w') as fo:
      fo.write(";;<<PT>><<LINE>>;;") # THIS LINE IS NEEDED FOR CUBE TO RECOGNIZE THIS AS A PT FILE
      fo.write('\n')

    # Using readlines()
    file1 = open(f, 'r')
    Lines = file1.readlines()
    
    last = Lines[-1]

    ctLINE = 0
    strNodesNewList = ""

    count = 0
    # Strips the newline character
    for line in Lines:
        count += 1
        if ctLINE==0: # write out all lines before first line that has ";"
          if line[0:1]==';' and line[0:2]!=';L' and count>1: #SKIP FIRST LINE SINCE MANUALLY CREATED PREVIOUSLY
            print (line)
            with open(strFilename, 'a') as fo:
              fo.write(line)

        #print("Line{}: {}".format(count, line.strip()))
        # find where N=
        posL = line.find("LINE NAME=")
        if posL>=0 or line is last: # LINE found since code only processes when another line is reached, so manually have to read last line
          if line is last:
            LINEstring = LINEstring + line.replace(";", "").strip()
          if ctLINE: # previously had been found, so this means that a new line is found
            newLINE = ""
            #print(LINEstring)

            # add code to change nodes

            # find where N=
            curPos = 0
            
            while curPos<len(LINEstring):
              posN = LINEstring.find("N=",curPos) # find the next N marker
              #print (strNodesNewList)
              

              if posN>=0:
                
                newLINE = newLINE + "," + LINEstring[curPos:posN] + "N="
                
                strNodesNewList = ""
                #print ("N Found at " + str(posN))
                curPos = posN + len("N=")

                while curPos<len(LINEstring): # if N foound then loop through rest of string until non-number found
                  # find next comma
                  posComma= LINEstring.find(",",curPos)
                  if posComma>0:
                    posNext = posComma
                  else:
                    posNext = len(LINEstring)
                
                  #print (posNext)
                  _sub = LINEstring[curPos:posNext].strip()
                  if _sub[0:1]=='-':
                    bNeg = True;
                  else:
                    bNeg = False;
                  curPos = posNext + 1
                  if _sub.strip('-').isnumeric(): # strip negative sign for isnumeric valuation, if it is numeric than this is a node number
                    iNode = _sub.strip('-')
                    
                    dfNodeNew = dfRenumber.loc[dfRenumber.N_v832_correct == int(iNode), 'N_V9']

                    if dfNodeNew.size>0:
                      iNodeNew = dfNodeNew.iloc[0]
                    else:
                      iNodeNew = 999999

                    if bNeg:
                      iNodeNew = iNodeNew * -1

                    strNodeNew = str(iNodeNew)
                    if len(strNodesNewList):
                      strNodesNewList = strNodesNewList + "," + strNodeNew
                    else:
                      strNodesNewList = strNodeNew
                    #print ('N v832: ' + _sub + ' N v9: ' + strNodeNew)
                  else: # if not a number than break out so that it can find the next N marker
                    #print ('OTHER:' + _sub)
                    newLINE = newLINE + strNodesNewList + "," + _sub
                    break
            #print (strNodesNewList)
            newLINE = newLINE + strNodesNewList
            newLINE = newLINE.strip(',')
            print(newLINE)
            with open(strFilename, 'a') as fo:
              fo.write(newLINE)
              fo.write('\n')

          #print (count)
          ctLINE = ctLINE + 1
          LINEstring = line.strip()
        elif (ctLINE): # continue appending LINE data if one has previously been found
          LINEstring = LINEstring + line.replace(";", "").strip()


FILE: E:\GitHub\TDM-Transit-Node-Renumbering-v832-to-v9\Transit-MAG-2050FC\Old Numbers\mag_brt_2050.lin
;************************************************************************************************

;Modes: Local bus=4, Enhanced bus (BRT I)=5, Express bus=6, Light rail=7, BRT=9, Commuter rail=8

;************************************************************************************************

;This file represents mountainland bus rapid transit service - 2050 Fiscally Constrained - RTP 19-50 Phase 3 - From MAG July 19, 2019

;************************************************************************************************

LINE NAME="BRT_ProOrm_50", LONGNAME="UVX", COLOR=3, MODE=9,ONEWAY=F, HEADWAY[1]=6, HEADWAY[2]=10, FARESYSTEM=4, N=66744,-66743,-66750,-66768,66770,-66769,-66771,-66775,66788,-66793,-66810,-66812,-66818,-66828,-66826,-66841,50024,-66841,-66826,-66825,-66837,-66874,66882,-66893,-66922,66941,-66966,-66985,67000,-67003,-67023,67041,-67061,-67060,67059,-67058,-67057,T